In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import utils
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
data = np.loadtxt('../input/data-first/train-1.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


In [6]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [7]:
def ToPos(turn, bias_x, bias_y):
    letter = ord(turn[0]) - ord('a')
    if 0 <= letter + bias_x <= 14:
        letter += bias_x
    else:
        return -1, -1
    
    num = int(turn[1:]) - 1
    if 0 <= num + bias_y <= 14:
        num += bias_y
    else:
        return -1, -1

    return letter, num

def Transform(game, color, turn, bias):
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard

    res = []
    labels = []

    for direction in range(8):
        if direction == 0:
            bias_x, bias_y = bias, 0
        if direction == 1:
            bias_x, bias_y = -bias, 0
        if direction == 2:
            bias_x, bias_y = 0, -bias
        if direction == 3:
            bias_x, bias_y = 0, bias
        if direction == 4:
            bias_x, bias_y = bias, bias
        if direction == 5:
            bias_x, bias_y = bias, -bias
        if direction == 6:
            bias_x, bias_y = -bias, -bias
        if direction == 7:
            bias_x, bias_y = -bias, bias

        curPlayer = 1
        whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
        blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
        for i in range(1, turn + 1):
            if len(game[i]) < 2:
                continue
            CurTurn = ToPos(game[i], bias_x, bias_y)
            if CurTurn[0] == -1 and CurTurn[1] == -1:
                continue
            if curPlayer == 1:
                blackBoard[CurTurn] = 1
            else:
                whiteBoard[CurTurn] = -1
            curPlayer *= -1

        curBoard = np.zeros(shape=(3, 15, 15))
        curBoard[0,:] = playerBoard
        curBoard[1,:] = blackBoard
        curBoard[2,:] = whiteBoard
        label = ToPos(game[turn + 1], bias_x, bias_y)
        if label[0] == -1 and label[1] == -1:
            continue

        res.append(curBoard)
        labels.append(label[0] * 15 + label[1])

    return np.asarray(res), np.asarray(labels)

In [8]:
NumOfFine = 0
for elem in data:
    if elem[0] != 'd' and elem[0] != 'u':
        NumOfFine += 1

In [9]:
Dataset = np.ndarray(shape=NumOfFine, dtype='U765')
idx = 0
for elem in data:
    if elem[0] != 'd' and elem[0] != 'u':
        Dataset[idx] = elem
        idx += 1
print(Dataset.shape)

(1941123,)


In [10]:
def GetRandom(data, bias):
    game = []
    while len(game) <= 4:
        gameNum = np.random.randint(data.shape[0])
        game = data[gameNum].split()
    color = game[0]
    turn = np.random.randint((len(game) - 2) // 2)
    if color[0] == 'b':
        board, label = toBoards(game, color, turn * 2, bias)
    else:
        board, label = toBoards(game, color, turn * 2 + 1, bias)

    return board, label

def Generator(size, data):
    Batch = np.ndarray(shape=(0, 3, 15, 15), dtype=np.int8)
    BatchLabels = np.ndarray(shape=0, dtype=np.long)
    for i in range(size):
        bias = np.random.randint(1, 3)
        res, labels = GetRandom(data, bias)
        Batch = np.concatenate((Batch, res), axis=0)
        BatchLabels = np.concatenate((BatchLabels, labels), axis=0)

    return torch.from_numpy(Batch).float(), torch.from_numpy(BatchLabels)

In [11]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(16, 1, kernel_size=5, stride=1, padding=2),
            nn.ReLU())

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(-1, 225)
        
        return x

In [12]:
net = ConvNet()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
CntBatch = 700000
CntEpoch = 2
BatchSize = 16
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [13]:
for epoch in range(CntEpoch):
    running_loss = 0.0
    
    for i in range(CntBatch):
        inputs, labels = Generator(BatchSize, Dataset)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 999))
            running_loss = 0.0
print('Finished Training')

[1,  1000] loss: 4.059
[1,  2000] loss: 3.482
[1,  3000] loss: 3.266
[1,  4000] loss: 3.128
[1,  5000] loss: 3.030
[1,  6000] loss: 2.951
[1,  7000] loss: 2.911
[1,  8000] loss: 2.860
[1,  9000] loss: 2.821
[1, 10000] loss: 2.811
[1, 11000] loss: 2.782
[1, 12000] loss: 2.747
[1, 13000] loss: 2.726
[1, 14000] loss: 2.690
[1, 15000] loss: 2.700
[1, 16000] loss: 2.693
[1, 17000] loss: 2.682
[1, 18000] loss: 2.645
[1, 19000] loss: 2.644
[1, 20000] loss: 2.632
[1, 21000] loss: 2.616
[1, 22000] loss: 2.612
[1, 23000] loss: 2.615
[1, 24000] loss: 2.585
[1, 25000] loss: 2.599
[1, 26000] loss: 2.597
[1, 27000] loss: 2.570
[1, 28000] loss: 2.558
[1, 29000] loss: 2.560
[1, 30000] loss: 2.551
[1, 31000] loss: 2.536
[1, 32000] loss: 2.538
[1, 33000] loss: 2.531
[1, 34000] loss: 2.513
[1, 35000] loss: 2.523
[1, 36000] loss: 2.496
[1, 37000] loss: 2.510
[1, 38000] loss: 2.496
[1, 39000] loss: 2.481
[1, 40000] loss: 2.473
[1, 41000] loss: 2.483
[1, 42000] loss: 2.475
[1, 43000] loss: 2.473
[1, 44000] 

In [14]:
torch.save(net.state_dict(), "./model5")